In [ ]:
%run ../env_variables.py

In [ ]:
import helpers.queries as q
import helpers.scopus_helpers as sh

In [ ]:
mergoni_scopus_step_0_df = sh.retrieve_results_from_list_of_queries(
    list_of_queries=[q.mergoni_scopus_query],
    file_name='mergoni_scopus_step_0',
    max_date='2021-01-01')

In [ ]:
mergoni_scopus_step_1_df = sh.retrieve_results_from_list_of_queries(
    list_of_queries=[q.mergoni_scopus_step_1_query],
    file_name='mergoni_scopus_step_1',
    max_date='2021-01-01')

In [ ]:
mergoni_scopus_step_2_df__weird = sh.retrieve_results_from_list_of_queries(
    list_of_queries=q.mergoni_scopus_step_2_queries_weird,
    file_name='mergoni_scopus_step_2__weird',
    max_date='2021-02-01')

In [ ]:
mergoni_scopus_step_2_df__non_weird = sh.retrieve_results_from_list_of_queries(
    list_of_queries=q.mergoni_scopus_step_2_queries_non_weird,
    file_name='mergoni_scopus_step_2__non_weird',
    max_date='2021-02-01')